# Lab_07: Classification


In [312]:
# !wget https://github.com/dbdmg/data-science-lab/raw/master/datasets/free-spoken-digit.zip -O free_spoken_digit.zip

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
import os
root = './free_spoken_digit/'
dev = root + 'dev/'
dev_samples = os.listdir(dev)


In [3]:
import scipy.io.wavfile as spwav

# Return the sample rate (in samples/sec) and data from an LPCM WAV file.
spwav.read(dev + dev_samples[0])

(8000, array([ 362,  360,  356, ..., -168,  -56,  318], dtype=int16))

In [4]:
def min_max_avg(ary: np.array) -> tuple:
    return (np.min(ary), np.max(ary), np.mean(ary))

In [5]:
d = dict()
frequency = spwav.read(dev + dev_samples[0])[0]
lengths = []
mma_list = [] # min_max_avg list
arys = []
for sample in dev_samples:
    id = int(sample.split('_')[0])
    ary = spwav.read(dev + sample)[1]
    time_length = ary.shape
    arys.append(ary)
    lengths.append(time_length)
    mma_list.append(min_max_avg(ary))
    d_temp = {'label': int(sample.split('_')[1][:-4]),
              'ary': ary,
              'min_max_avg':min_max_avg(ary),
              'time': round(time_length[0]/frequency, 2)} # till -4 since it has the .wav extension
    d[id] = d_temp 
d

{481: {'label': 8,
  'ary': array([ 362,  360,  356, ..., -168,  -56,  318], dtype=int16),
  'min_max_avg': (-11989, 7846, 0.09351203167834297),
  'time': 0.41},
 863: {'label': 1,
  'ary': array([ 3,  6, -2, ..., 13,  6,  6], dtype=int16),
  'min_max_avg': (-2957, 1874, -1.7005242463958061),
  'time': 0.38},
 845: {'label': 2,
  'ary': array([-21,  31, -34, ...,  -4,   9,  34], dtype=int16),
  'min_max_avg': (-1227, 1604, 0.10849233071455294),
  'time': 0.33},
 1146: {'label': 3,
  'ary': array([  0,   8, -17, ...,  19,   9,   2], dtype=int16),
  'min_max_avg': (-490, 592, -0.06704980842911877),
  'time': 0.33},
 307: {'label': 2,
  'ary': array([   0,    0, -256, ..., -512, -512, -768], dtype=int16),
  'min_max_avg': (-4608, 4608, -230.59830320914793),
  'time': 0.34},
 1281: {'label': 5,
  'ary': array([  0,   0, -14, ..., -12,  -8, -13], dtype=int16),
  'min_max_avg': (-4278, 2311, 0.471718835804503),
  'time': 0.46},
 223: {'label': 6,
  'ary': array([-2,  0, -2, ..., 16, 17, -6],

The method `read()` of `scipy.io.wavfile` returns:
- the sample rate in sample/sec (in our case 8000, hence the frequency is 8kHz)
- `numpy array of dtype=int16`. According to documentation, it is a format of 16-bit integer PCM with min = -32767 and max = 32767

In [6]:
for length in lengths:
    print(round(length[0]/8000, 2), '\tsec')

0.41 	sec
0.38 	sec
0.33 	sec
0.33 	sec
0.34 	sec
0.46 	sec
0.27 	sec
0.34 	sec
0.32 	sec
0.22 	sec
0.48 	sec
0.25 	sec
0.47 	sec
0.32 	sec
0.34 	sec
0.39 	sec
0.37 	sec
0.37 	sec
0.69 	sec
0.4 	sec
0.41 	sec
0.4 	sec
0.28 	sec
0.54 	sec
0.31 	sec
0.41 	sec
0.33 	sec
0.23 	sec
0.6 	sec
0.51 	sec
0.33 	sec
0.42 	sec
0.36 	sec
0.25 	sec
0.42 	sec
0.61 	sec
0.5 	sec
0.37 	sec
0.45 	sec
0.6 	sec
0.66 	sec
0.43 	sec
0.46 	sec
0.25 	sec
0.49 	sec
0.52 	sec
0.63 	sec
0.5 	sec
0.26 	sec
0.41 	sec
0.28 	sec
0.39 	sec
0.47 	sec
0.33 	sec
0.48 	sec
0.38 	sec
0.37 	sec
0.3 	sec
0.45 	sec
0.51 	sec
0.18 	sec
0.58 	sec
0.44 	sec
0.72 	sec
0.37 	sec
0.43 	sec
0.21 	sec
0.41 	sec
0.36 	sec
0.47 	sec
0.36 	sec
0.27 	sec
0.39 	sec
0.35 	sec
0.58 	sec
0.27 	sec
0.37 	sec
0.48 	sec
0.53 	sec
0.3 	sec
0.49 	sec
0.17 	sec
0.39 	sec
0.41 	sec
0.16 	sec
0.34 	sec
0.29 	sec
0.44 	sec
0.35 	sec
0.44 	sec
0.21 	sec
0.49 	sec
0.26 	sec
0.54 	sec
0.2 	sec
0.28 	sec
0.51 	sec
0.33 	sec
0.28 	sec
0.51 	sec
0.49 	sec

As we can see, all the samples are below 1 sec length but they are not aligned (they don't have the same length). If we want to align them, we can paddle each array with a neutral value.
I now want to check whether if conditions where the same. The idea that I've got is to check min, max and avg of each sample.

In [8]:
print('min\tmax\tavg')
medie = []
for mma in mma_list:
    print(f'{mma[0]}\t{mma[1]}\t{round(mma[2],2)}')
    medie.append(round(mma[2],2))

min	max	avg
-11989	7846	0.09
-2957	1874	-1.7
-1227	1604	0.11
-490	592	-0.07
-4608	4608	-230.6
-4278	2311	0.47
-2603	1449	-0.83
-12032	8960	-229.93
-1188	906	-0.19
-519	833	-0.4
-13391	9945	-0.34
-7680	4864	-229.34
-2877	1884	0.06
-5120	3072	-230.7
-1885	2069	0.08
-3059	1272	-0.85
-7936	6656	-231.37
-1782	1141	-0.58
-25254	23473	-0.16
-10183	10571	0.19
-556	925	-0.15
-14520	9727	-0.46
-8192	5376	-231.85
-10516	8765	0.6
-5888	3328	-229.09
-14776	13650	0.29
-3832	1704	0.35
-6656	4608	-230.37
-15747	17403	0.3
-12407	10085	-0.33
-668	835	0.16
-13824	7424	-251.46
-8192	6400	-231.17
-4352	4608	-230.31
-783	834	-0.1
-805	994	0.11
-9216	6144	-253.75
-7936	7168	-229.54
-1808	1157	-0.55
-11205	7531	-1.13
-10605	8792	0.52
-15871	16045	0.36
-874	1086	-0.12
-730	868	-0.34
-14182	12363	-0.74
-25136	31504	0.08
-24071	24044	0.36
-845	664	-0.08
-2304	2048	-229.45
-566	803	0.13
-1117	973	-1.08
-1836	1130	-0.25
-13891	12911	0.66
-8960	5376	-228.02
-11530	9447	-0.67
-8960	5632	-227.91
-8448	5376	-248.4
-28

In [18]:
medie = np.array(medie)
mask = medie < -5
len(medie[mask])

372

There are some values where the average is strongly negative, if we try to hear them we can see that they are noisier than the others.

_We can try to remove them and then classify_

For what concern the alignment, since I don't want to change the average, I shall add 0 as padding element.

In [19]:
# I will align them to such value
max_length = max(lengths)[0]
max_length

17567

In [20]:
# Check if all samples are 1D
for l in lengths:
    if len(l) != 1:
        print(l)
# They all have the same shape

In [21]:
def padding(ary: np.array, target_len:int, padding_element:int=0) -> np.array:
    return np.pad(ary, (0, target_len), 'constant',  constant_values=padding_element)

In [22]:
new_arys = []
for a in arys:
    width_to_pad = max_length-len(a)
    new_ar = padding(a, width_to_pad, 0)
    #new_ar = padding(a, width_to_pad, np.mean(a)) #try with this one
    new_arys.append(new_ar)

what if we pad with mean?


In [23]:
# Check if they are all padded with the same length
for array in new_arys:
    if len(array) != max_length:
        print('Found one')

In [24]:
new_mma_list = []
for a in new_arys:
    new_mma_list.append(min_max_avg(a))

print('min\tmax\tavg')
for mma in new_mma_list:
    print(f'{mma[0]}\t{mma[1]}\t{round(mma[2],2)}')

min	max	avg
-11989	7846	0.02
-2957	1874	-0.3
-1227	1604	0.02
-490	592	-0.01
-4608	4608	-35.59
-4278	2311	0.1
-2603	1449	-0.1
-12032	8960	-35.09
-1188	906	-0.03
-519	833	-0.04
-13391	9945	-0.07
-7680	4864	-26.58
-2877	1884	0.01
-5120	3072	-33.49
-1885	2069	0.01
-3059	1272	-0.15
-7936	6656	-38.6
-1782	1141	-0.1
-25254	23473	-0.05
-10183	10571	0.03
-556	925	-0.03
-14520	9727	-0.08
-8192	5376	-29.52
-10516	8765	0.15
-5888	3328	-32.38
-14776	13650	0.05
-3832	1704	0.05
-6656	4608	-24.37
-15747	17403	0.08
-12407	10085	-0.08
-668	835	0.02
-13824	7424	-48.38
-8192	6400	-37.86
-4352	4608	-26.52
-783	834	-0.02
-805	994	0.03
-9216	6144	-57.4
-7936	7168	-38.56
-1808	1157	-0.11
-11205	7531	-0.31
-10605	8792	0.16
-15871	16045	0.07
-874	1086	-0.03
-730	868	-0.04
-14182	12363	-0.17
-25136	31504	0.02
-24071	24044	0.1
-845	664	-0.02
-2304	2048	-26.83
-566	803	0.02
-1117	973	-0.14
-1836	1130	-0.04
-13891	12911	0.14
-8960	5376	-34.44
-11530	9447	-0.15
-8960	5632	-39.73
-8448	5376	-41.42
-2816	2304	-31.56
-

**NOTE**: If we pad with 0, then if we didn't remove the strongly negative values their value will go towards 0 mean. Otherwise if we pad using mean value, the mean value remains the same. What can we do? We can remove strongly negative mean values before and then apply the same preprocessing or we can add the mean value. By now I don't know what is right.

NOTA: provare bucketing in cui inserisco un parametro per vedere in quanti bucket dividere. Dopodiché usare una moving average come valore di ogni bucket. Provare con moving average in cui al bucket 1 media di bucket 1, bucket 2 media dei valori fino al termine di bucket 2, e così via.
Un'alternativa è usare la exponential moving average (guarda wikipedia)

In [25]:
df = pd.DataFrame.from_dict(d).T
df.ary = new_arys
df.min_max_avg = new_mma_list
df.sort_index(axis=0, inplace=True)
df.head()

,label,ary,min_max_avg,time
0,4,"[14, 21, 0, 18, -11, 22, -15, -53, 9, -14, 11,...","(-8198, 10350, -0.009221836397791313)",0.39
1,7,"[-2, 12, 1, -18, 7, -7, 1, -1, 0, 0, -6, -9, 1...","(-1885, 1110, -0.02134684351340582)",0.41
2,5,"[-9, -7, -5, -8, -13, -1, 20, -12, 6, 15, 3, 2...","(-3789, 1686, -0.0987078044059885)",0.42
3,2,"[-256, -512, 0, -256, 256, -256, 0, -512, 0, -...","(-4864, 4096, -35.615870666590766)",0.34
4,3,"[-20, 10, 26, -13, 22, -16, 11, 7, -21, 1, 63,...","(-566, 835, 0.0005692491603574884)",0.24


In [26]:
avg = []
for tup in df.min_max_avg:
    avg.append(tup[-1])
df['avg'] = avg
df.head()

,label,ary,min_max_avg,time,avg
0,4,"[14, 21, 0, 18, -11, 22, -15, -53, 9, -14, 11,...","(-8198, 10350, -0.009221836397791313)",0.39,-0.009222
1,7,"[-2, 12, 1, -18, 7, -7, 1, -1, 0, 0, -6, -9, 1...","(-1885, 1110, -0.02134684351340582)",0.41,-0.021347
2,5,"[-9, -7, -5, -8, -13, -1, 20, -12, 6, 15, 3, 2...","(-3789, 1686, -0.0987078044059885)",0.42,-0.098708
3,2,"[-256, -512, 0, -256, 256, -256, 0, -512, 0, -...","(-4864, 4096, -35.615870666590766)",0.34,-35.615871
4,3,"[-20, 10, 26, -13, 22, -16, 11, 7, -21, 1, 63,...","(-566, 835, 0.0005692491603574884)",0.24,0.000569


In [28]:
df1 = df[df.avg > - 5] # remove noisy points
grpby_label = df1.groupby(['label'])
grpby_label.describe()

avg                                                              \
       count      mean       std       min       25%       50%       75%   
label                                                                      
0      115.0 -0.002679  0.132308 -0.342916 -0.067342 -0.013207  0.053310   
1      111.0 -0.051713  0.112771 -0.566061 -0.110491 -0.035464  0.006945   
2      111.0 -0.030483  0.080815 -0.271703 -0.071270 -0.023852  0.015057   
3      112.0 -0.042779  0.094218 -0.318210 -0.097242 -0.030512  0.011883   
4      113.0  0.041899  0.134809 -0.248933 -0.029146  0.004099  0.080947   
5      115.0 -0.025799  0.128149 -0.480048 -0.069363 -0.014459  0.026299   
6      110.0 -0.034165  0.094040 -0.322593 -0.088860 -0.024848  0.020849   
7      111.0 -0.047581  0.121456 -0.677463 -0.092873 -0.030398  0.024791   
8      116.0 -0.030431  0.072725 -0.306598 -0.068808 -0.026299  0.014786   
9      114.0 -0.023887  0.097420 -0.227700 -0.087679 -0.011613  0.033956   

                 
            max  
label            
0      0.430979  
1      0.291854  
2      0.258553  
3      0.195366  
4      0.502249  
5      0.545568  
6      0.183640  
7      0.174532  
8      0.205100  
9      0.312745

The number of samples for each label is similar, hence we can proceed

In [329]:
mask = df.label == 2
df[mask]

,label,ary,min_max_avg,time,avg
3,2,"[-256, -512, 0, -256, 256, -256, 0, -512, 0, -...","(-4864, 4096, -35.615870666590766)",0.34,-35.615871
17,2,"[-256, -256, -256, -256, -256, -256, -256, -25...","(-6144, 4864, -37.13143963112655)",0.36,-37.131440
39,2,"[-942, 867, -154, 123, 280, -402, 991, -654, 2...","(-8931, 7994, -0.01616667615415267)",0.51,-0.016167
60,2,"[-59, -16, -32, 8, 3, 18, 15, 16, 6, -1, 10, 5...","(-2035, 1308, -0.15130642682302042)",0.31,-0.151306
67,2,"[0, 0, 2, -2, 2, -2, 2, -3, 4, -4, 0, 0, -3, 0...","(-2665, 1363, -0.1060511185746001)",0.33,-0.106051
...,...,...,...,...,...
1460,2,"[-5, -6, -8, -18, 4, -7, 2, 0, -5, -2, -2, -14...","(-642, 702, 0.003017020549894689)",0.27,0.003017
1467,2,"[3, -3, 7, 0, 6, -9, 6, 2, 12, 4, 7, 8, -9, 3,...","(-917, 1330, -0.025388512551943986)",0.24,-0.025389
1474,2,"[-6, 8, -4, 9, 3, 12, 6, 14, -15, 6, -1, 15, 9...","(-530, 744, 0.02117606876529857)",0.27,0.021176
1477,2,"[-12, 6, 11, 3, -1, 4, -2, -1, -2, 2, -5, -1, ...","(-602, 729, -0.06728525075425514)",0.53,-0.067285


In [330]:
#mask1 = df.avg < -10 # before zero padding. Ok with mean padding
#mask1 = df.avg < - 5 # this is good with 0 padding. Still 372 lines
df = df[df.avg > - 5]
df

,label,ary,min_max_avg,time,avg
0,4,"[14, 21, 0, 18, -11, 22, -15, -53, 9, -14, 11,...","(-8198, 10350, -0.009221836397791313)",0.39,-0.009222
1,7,"[-2, 12, 1, -18, 7, -7, 1, -1, 0, 0, -6, -9, 1...","(-1885, 1110, -0.02134684351340582)",0.41,-0.021347
2,5,"[-9, -7, -5, -8, -13, -1, 20, -12, 6, 15, 3, 2...","(-3789, 1686, -0.0987078044059885)",0.42,-0.098708
4,3,"[-20, 10, 26, -13, 22, -16, 11, 7, -21, 1, 63,...","(-566, 835, 0.0005692491603574884)",0.24,0.000569
6,1,"[2, 17, 16, 23, 25, 19, 26, 10, 17, 24, 6, 6, ...","(-1858, 2166, -0.022428416918085045)",0.44,-0.022428
...,...,...,...,...,...
1495,7,"[-2, 0, 5, 4, 5, 7, 4, 4, 2, -12, -20, -19, -2...","(-3068, 1509, -0.3677918825069733)",0.33,-0.367792
1496,4,"[-3, -17, -1, 15, 5, 2, 8, 8, 3, 5, -9, -8, -1...","(-2935, 2249, -0.014174304092901463)",0.39,-0.014174
1497,4,"[319, 293, 229, 200, 199, 205, 245, 251, 246, ...","(-12973, 15004, -0.08106108043490635)",0.44,-0.081061
1498,7,"[-312, -273, -194, -228, -123, -103, -108, -93...","(-3998, 5326, 0.08100415551887061)",0.41,0.081004


there are around 370 samples with pretty low avg. 372/1500 are very noisy.

# Bucketing

In [331]:
# portion = .10
# num_buckets = round(portion * max_length)
num_buckets = 10 #divided into 10 buckets

In [332]:
prova = np.array_split(df.ary[0], num_buckets)
cnt = 0
for a in prova:
    print(np.mean(a))
    cnt += 1
# print(cnt)

-0.5167899829254411
0.42458736482640863
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [333]:
# We want to divide each array in a set of features.
# To do this, we create a new dictionary in which we will have
# the id and the set of features.
# By changing the parameter num_buckets we can increase or decrease
# the number of features that we will use

# portion = .10
# num_buckets = round(portion * max_length)
num_buckets = 1000 #divided into num_buckets buckets

another_dict = dict()

for id, array in zip(df.ary.index, df.ary):
    split_ary = np.array_split(array, num_buckets)
    cnt_feature = 0
    features = dict()
    for subary in split_ary:
        features[f'feature_{cnt_feature}'] = subary
        cnt_feature += 1
    another_dict[id] = features

another_dict[0]

{'feature_0': array([ 14,  21,   0,  18, -11,  22, -15, -53,   9, -14,  11,  11,  27,
         14,  12,  29,  39,   5], dtype=int16),
 'feature_1': array([ -8, -23, -19, -22,  -9,   0,   8,   1,  12,   1,  25,  46, -28,
         37, -43,  31, -78, -18], dtype=int16),
 'feature_2': array([ -23,  -34,   26,  -26,   52,  -29,   89,   -3,   71,  -47,   57,
         -83,   88,  171, -186,   14,  -35,   -5], dtype=int16),
 'feature_3': array([   2,  -15,   64,  -16,   14,  -57,   51,  171, -219,   55,  -57,
         -11,   37,    3,   64,  -22,   84,  -77], dtype=int16),
 'feature_4': array([151, -78,  75, 102,  -5,  37, -91,  -2, -45,  91, -81,  -2, -15,
         51, 118, -17,  29, -41], dtype=int16),
 'feature_5': array([ 111, -109,   34,  189,  175,  -88,   -5,  -45,  -56,   -8, -110,
         134, -347, -157, -148, -145,  -90,  -31], dtype=int16),
 'feature_6': array([ -91,  -50,   30,  253,  287,  -63,  142,   57,   51, -136,   18,
          66,   -9, -300, -121,   36,  -12,   49], dtyp

In [334]:
def simple_moving_average(list_of_arys:list) -> np.array:
    list_of_arys = list(list_of_arys)
    ary = np.zeros(shape=(len(list_of_arys)))
    for id in range(len(list_of_arys)):
        num_values = len(list_of_arys[id])
        tot = np.sum(list_of_arys[id])
        ary[id] = tot/num_values
    return ary

def cumulative_moving_average(list_of_arys:list) -> np.array:
    temp = [(np.sum(i), len(i)) for i in list_of_arys]
    mean_variable = 0
    values_cnt = 0
    out_list = []
    
    for i in range(len(temp)):
        sum, l = temp[i]
        if i == 0:
            mean_variable = sum/l
            values_cnt += l
            out_list.append(mean_variable)
        else:
            mean_variable = (sum + values_cnt*mean_variable)/(l + values_cnt)
            values_cnt += l
            out_list.append(mean_variable)


    return np.array(out_list)

### Simple moving average (SMA)

In [335]:
# SMA
sma_dict = {}
for key, item in another_dict.items():
    simple_mean = simple_moving_average(item.values())
    sma_dict[key] = {f'feature_{i}':m for i,m in enumerate(simple_mean)}
sma_dict[10]

{'feature_0': -11.0,
 'feature_1': 14.666666666666666,
 'feature_2': -8.777777777777779,
 'feature_3': -1.1111111111111112,
 'feature_4': 8.5,
 'feature_5': -9.833333333333334,
 'feature_6': -0.3333333333333333,
 'feature_7': 10.555555555555555,
 'feature_8': -18.333333333333332,
 'feature_9': 9.277777777777779,
 'feature_10': 6.666666666666667,
 'feature_11': -1.8888888888888888,
 'feature_12': -15.0,
 'feature_13': 26.11111111111111,
 'feature_14': -28.22222222222222,
 'feature_15': -90.05555555555556,
 'feature_16': 87.22222222222223,
 'feature_17': 53.0,
 'feature_18': -251.11111111111111,
 'feature_19': 37.05555555555556,
 'feature_20': 136.11111111111111,
 'feature_21': -41.55555555555556,
 'feature_22': -74.94444444444444,
 'feature_23': 63.44444444444444,
 'feature_24': 119.27777777777777,
 'feature_25': -248.0,
 'feature_26': 24.11111111111111,
 'feature_27': 150.88888888888889,
 'feature_28': 100.16666666666667,
 'feature_29': -211.05555555555554,
 'feature_30': 40.5,
 'featu

### Cumulative moving average
In a cumulative average (CA), the data arrive in an ordered datum stream, and the user would like to get the average of all of the data up until the current datum. For example, an investor may want the average price of all of the stock transactions for a particular stock up until the current time. As each new transaction occurs, the average price at the time of the transaction can be calculated for all of the transactions up to that point using the cumulative average, typically an equally weighted average of the sequence of _n_ values $x_{1}, \ldots ,x_{n}$ up to the current time:

$$
CA_n = \frac{x_1+\dots+x_n}{n}
$$

it is possible to simply update cumulative average as a new value, 
$x_{n+1}$ becomes available, using the formula:

$$
CA_n = \frac{x_{n+1}+n\cdot CA_n}{n+1} \to CA_n = \frac{x_{n+1} + \dots + x_{n+n_1}+n\cdot CA_n}{n+n_1}
$$

In [336]:
# Cumulative average
cum_avg_dict = dict()
for key, item in another_dict.items():
    mean = cumulative_moving_average(item.values())        
    cum_avg_dict[key] = {f'feature_{i}': m for i,m in enumerate(mean)}
cum_avg_dict[10]

{'feature_0': -11.0,
 'feature_1': 1.8333333333333333,
 'feature_2': -1.7037037037037037,
 'feature_3': -1.5555555555555556,
 'feature_4': 0.45555555555555555,
 'feature_5': -1.2592592592592593,
 'feature_6': -1.126984126984127,
 'feature_7': 0.3333333333333333,
 'feature_8': -1.7407407407407407,
 'feature_9': -0.6388888888888888,
 'feature_10': 0.025252525252525325,
 'feature_11': -0.1342592592592592,
 'feature_12': -1.2777777777777777,
 'feature_13': 0.6785714285714286,
 'feature_14': -1.2481481481481482,
 'feature_15': -6.798611111111111,
 'feature_16': -1.2679738562091503,
 'feature_17': 1.7469135802469136,
 'feature_18': -11.56140350877193,
 'feature_19': -9.130555555555556,
 'feature_20': -2.2142857142857153,
 'feature_21': -4.0025252525252535,
 'feature_22': -7.086956521739132,
 'feature_23': -4.148148148148149,
 'feature_24': 0.7888888888888884,
 'feature_25': -8.77991452991453,
 'feature_26': -7.561728395061729,
 'feature_27': -1.9027777777777777,
 'feature_28': 1.616858237547

### Exponential weighted average


In [337]:
# TODO

------------------

In [338]:
df.label

0       4
1       7
2       5
4       3
6       1
       ..
1495    7
1496    4
1497    4
1498    7
1499    7
Name: label, Length: 1128, dtype: object

In [339]:
cum_df = pd.DataFrame.from_dict(cum_avg_dict).T
cum_df['label'] = df.label
cum_df.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_991,feature_992,feature_993,feature_994,feature_995,feature_996,feature_997,feature_998,feature_999,label
0,7.722222,1.444444,2.759259,3.055556,5.522222,-1.842593,0.063492,-0.909722,-21.191358,20.900000,...,-0.009294,-0.009285,-0.009276,-0.009267,-0.009258,-0.009249,-0.009240,-0.009231,-0.009222,4
1,-0.944444,-0.555556,-0.722222,-0.666667,-0.655556,-0.805556,-0.611111,-0.611111,-0.407407,-0.566667,...,-0.021513,-0.021492,-0.021472,-0.021451,-0.021430,-0.021409,-0.021388,-0.021368,-0.021347,7
2,1.833333,-1.416667,-0.944444,-2.180556,-1.766667,-2.101852,-1.928571,-1.791667,-1.549383,-1.533333,...,-0.099478,-0.099381,-0.099284,-0.099188,-0.099091,-0.098995,-0.098899,-0.098803,-0.098708,5
4,2.611111,2.972222,8.481481,0.305556,-4.288889,-3.037037,-2.984127,-2.388889,-2.228395,-1.338889,...,0.000574,0.000573,0.000573,0.000572,0.000571,0.000571,0.000570,0.000570,0.000569,3
6,10.277778,-16.361111,9.092593,17.111111,-14.422222,7.620370,-7.547619,3.159722,11.148148,-14.061111,...,-0.022603,-0.022581,-0.022559,-0.022537,-0.022516,-0.022494,-0.022472,-0.022450,-0.022428,1


In [340]:
sma_df = pd.DataFrame.from_dict(sma_dict).T
sma_df['label'] = df.label
sma_df.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_991,feature_992,feature_993,feature_994,feature_995,feature_996,feature_997,feature_998,feature_999,label
0,7.722222,-4.833333,5.388889,3.944444,15.388889,-38.666667,11.500000,-7.722222,-183.444444,399.722222,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
1,-0.944444,-0.166667,-1.055556,-0.500000,-0.611111,-1.555556,0.555556,-0.611111,1.222222,-2.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
2,1.833333,-4.666667,0.000000,-5.888889,-0.111111,-3.777778,-0.888889,-0.833333,0.388889,-1.388889,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
4,2.611111,3.333333,19.500000,-24.222222,-22.666667,3.222222,-2.666667,1.777778,-0.944444,6.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
6,10.277778,-43.000000,60.000000,41.166667,-140.555556,117.833333,-98.555556,78.111111,75.055556,-240.944444,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [341]:
sma_df[sma_df.columns[:-1]]

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_990,feature_991,feature_992,feature_993,feature_994,feature_995,feature_996,feature_997,feature_998,feature_999
0,7.722222,-4.833333,5.388889,3.944444,15.388889,-38.666667,11.500000,-7.722222,-183.444444,399.722222,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.944444,-0.166667,-1.055556,-0.500000,-0.611111,-1.555556,0.555556,-0.611111,1.222222,-2.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.833333,-4.666667,0.000000,-5.888889,-0.111111,-3.777778,-0.888889,-0.833333,0.388889,-1.388889,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.611111,3.333333,19.500000,-24.222222,-22.666667,3.222222,-2.666667,1.777778,-0.944444,6.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,10.277778,-43.000000,60.000000,41.166667,-140.555556,117.833333,-98.555556,78.111111,75.055556,-240.944444,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,-5.444444,0.777778,16.277778,-39.277778,20.888889,39.333333,-349.944444,0.055556,295.277778,-385.222222,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1496,-0.055556,0.166667,-6.277778,4.277778,-0.277778,-2.388889,1.888889,-2.055556,-0.555556,-1.944444,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1497,129.166667,-409.444444,-21.722222,373.000000,-1329.444444,343.944444,536.444444,590.611111,-1910.055556,820.944444,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1498,-265.777778,381.166667,-27.444444,-28.611111,15.666667,-11.666667,-657.277778,470.222222,426.611111,-151.277778,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### SMA

In [360]:
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

X_train, X_test, y_train, y_test = train_test_split(sma_df[sma_df.columns[:-1]], sma_df[sma_df.columns[-1]], test_size=.2)

y_train = y_train.astype('int')
y_test = y_test.astype('int')
clf_sma = RandomForestClassifier().fit(X_train, y_train)
preds = clf_sma.predict(X_test)
accuracy_score(y_test, preds)
f1_score(y_test, preds, average='macro')

0.43272043654997516

In [374]:
from sklearn.neighbors import KNeighborsClassifier
knn_sma = KNeighborsClassifier(n_neighbors=4).fit(X_train, y_train)
preds = knn_sma.predict(X_test)
f1_score(y_test, preds, average='macro')

0.17260145414607658

In [375]:
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

X_train, X_test, y_train, y_test = train_test_split(cum_df[cum_df.columns[:-1]], cum_df[cum_df.columns[-1]], test_size=.2)

y_train = y_train.astype('int')
y_test = y_test.astype('int')
clf_cum = RandomForestClassifier().fit(X_train, y_train)
preds = clf_cum.predict(X_test)
accuracy_score(y_test, preds)
f1_score(y_test, preds, average='macro')

0.3080159348147079

In [387]:
from sklearn.neighbors import KNeighborsClassifier
knn_cum = KNeighborsClassifier(n_neighbors=25).fit(X_train, y_train)
preds = knn_cum.predict(X_test)
f1_score(y_test, preds, average='macro')

0.19053421842365098

IDEA, si può provare a usare cma e sma insieme (al dataframe di una vengono aggiunte le features dell'altro). Però diventerebbe molto complicato. Al momento bisogna riorganizzare tutto e scrivere qualche funzione che consenta di sfruttare paramgrid e pulisca il codice.
- fare parte preprocessing 
  - inserire l'hyperparameter per il bucketing
  - generare funzioni che creino il dataframe con cma e sma
- usare i classificatori

Provare a fare un modello ensemble potrebbe aiutare? Forse. 